In [42]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import open3d as o3d
from shapely import LineString
from shapely.geometry import Point
from typing import List, Dict, Any
import os

First, we are going to simply concat all of our datafiles together, just to make life easy

In [49]:
folder_path = 'drone_flight_dataset_dedrone'
all_files = os.listdir(folder_path)
csv_files = [file for file in all_files if file.endswith('.csv')]

all_df = []

for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    all_df.append(df)

df = pd.concat(all_df, join="inner")

In [58]:
df

,DetectionLatitude,DetectionLongitude,Altitude,SensorTime,DroneType,DroneId,Coordinates
0,49.277390,-123.145985,58.8,2023-05-03 16:42:17,Mavic Air 2,eab17604417cd77a1c8c3de264d26104,POINT Z (-123.145985065228 49.2773898688311 58.8)
1,49.277390,-123.145985,58.1,2023-05-03 16:42:30,Mavic Air 2,eab17604417cd77a1c8c3de264d26104,POINT Z (-123.145985065228 49.2773898688311 58.1)
2,49.277390,-123.145985,58.4,2023-05-03 16:42:33,Mavic Air 2,eab17604417cd77a1c8c3de264d26104,POINT Z (-123.145985065228 49.2773898688311 58.4)
3,49.277390,-123.145985,58.4,2023-05-03 16:42:35,Mavic Air 2,eab17604417cd77a1c8c3de264d26104,POINT Z (-123.145985065228 49.2773898688311 58.4)
4,49.277390,-123.145985,58.3,2023-05-03 16:42:49,Mavic Air 2,eab17604417cd77a1c8c3de264d26104,POINT Z (-123.145985065228 49.2773898688311 58.3)
...,...,...,...,...,...,...,...
272743,42.225901,-70.869981,38.3,2023-05-01 16:41:22,Mini 3 Pro,6041ac7a05ef59ed1d65eb32e92ff0a8,POINT Z (-70.86998054951522 42.22590092958105 ...
272744,42.225901,-70.869981,38.3,2023-05-01 16:41:40,Mini 3 Pro,6041ac7a05ef59ed1d65eb32e92ff0a8,POINT Z (-70.86998054951522 42.22590092958105 ...
272745,42.225901,-70.869981,38.4,2023-05-01 16:41:44,Mini 3 Pro,6041ac7a05ef59ed1d65eb32e92ff0a8,POINT Z (-70.86998054951522 42.22590092958105 ...
272746,42.225901,-70.869981,38.5,2023-05-01 16:41:52,Mini 3 Pro,6041ac7a05ef59ed1d65eb32e92ff0a8,POINT Z (-70.86998054951522 42.22590092958105 ...


In [51]:
# Cleaning our total df. All we are doing is creating a point column
df['SensorTime'] = pd.to_datetime(df['SensorTime'], format='mixed')
df['Coordinates'] = list(zip(df.DetectionLongitude, df.DetectionLatitude, df.Altitude))
df['Coordinates'] = df['Coordinates'].apply(Point)

drone_ids = df['DroneId'].unique()
print(len(drone_ids))

27977


Here, we are creating our flight dictionary, which contains flattened maps of our flights

In [ ]:
# Flghts is going to be a list of hashmaps, which we will later form to a dataframe (i am pandas amateur)
flights: List[Dict[str, Any]] = []
flight_id = 0


for drone_id in drone_ids:
    filtered_df = df[df['DroneId'] == drone_id]
    filtered_df = filtered_df.sort_values(by='SensorTime')
    
    # we make an assumption that we have a differenet flight if over 30 minutes
    filtered_df['gap'] = filtered_df['SensorTime'].diff() > pd.Timedelta(hours=0.33)
    assert filtered_df['gap'].iloc[0] == False
    filtered_df['group'] = filtered_df['gap'].cumsum()
    datagrams = [v for k, v in filtered_df.groupby('group')]
    
    # each flght for a given drone id 
    for d in datagrams:
        if len(d.index) <= 10:
            continue
           
        # we create a coordinate string and line string 
        path = LineString(list(d['Coordinates']))
        times = list(d['SensorTime'])
        
        frame = {
            'DroneId': drone_id, 
            'flight_id': flight_id,
            'times': times,
            'path': path
        }
        flight_id += 1
        
        flights.append(frame)

In [ ]:
processed = gpd.GeoDataFrame(flights)

In [57]:
processed

,DroneId,flight_id,times,path
0,eab17604417cd77a1c8c3de264d26104,0,"[2023-05-03 14:38:31, 2023-05-03 14:38:48, 202...",LINESTRING Z (-123.112043045445 49.27908009432...
1,eab17604417cd77a1c8c3de264d26104,1,"[2023-05-03 15:45:08, 2023-05-03 15:45:12, 202...",LINESTRING Z (-123.121800516696 49.26673285384...
2,eab17604417cd77a1c8c3de264d26104,2,"[2023-05-03 16:42:17, 2023-05-03 16:42:30, 202...",LINESTRING Z (-123.145985065228 49.27738986883...
3,cb306d53d8b752b0055e1f830cabb966,3,"[2023-05-03 15:01:06, 2023-05-03 15:01:06, 202...",LINESTRING Z (-97.9479658664172 26.20781720568...
4,3774ab0ffcc949cfe8a33805623e5ba1,4,"[2023-05-03 19:47:27, 2023-05-03 19:47:31, 202...",LINESTRING Z (37.1785609654188 47.740121186183...
...,...,...,...,...
73,e59b1527164622a8c22d7c89c76f1bd7,73,"[2023-05-03 12:20:58, 2023-05-03 12:20:58, 202...",LINESTRING Z (-81.4971500864194 28.39329023069...
74,a79856912e0c53847df21e13775fd0c5,74,"[2023-05-03 16:20:13, 2023-05-03 16:20:17, 202...",LINESTRING Z (-112.418404593747 33.72137373664...
75,a79856912e0c53847df21e13775fd0c5,75,"[2023-05-03 16:51:08, 2023-05-03 16:51:14, 202...",LINESTRING Z (-112.179315035417 33.77407439464...
76,a79856912e0c53847df21e13775fd0c5,76,"[2023-05-03 19:51:08, 2023-05-03 19:51:08, 202...",LINESTRING Z (-112.179315035417 33.77406866506...


In [ ]:
 # average altitue decreae